In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression  # <-- CRITICAL LINE
from scipy.linalg import khatri_rao
from sklearn.svm import LinearSVC
import time as tm
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['lbfgs', 'newton-cg', 'liblinear', 'sag', 'saga'],
    'max_iter': [1000, 2500, 5000]
}

In [ ]:
def my_fit( X_train, y_train ):
################################
#  Non Editable Region Ending  #
################################

	# Use this method to train your models using training CRPs
	# X_train has 8 columns containing the challenge bits
	# y_train contains the values for responses

	# THE RETURNED MODEL SHOULD BE ONE VECTOR AND ONE BIAS TERM
	# If you do not wish to use a bias term, set it to 0
	# return w, b
    # Map the challenges to the ML-PUF feature space
    X_feat = my_map(X_train)

    # Use GridSearchCV for optimal hyperparameter selection
    #param_grid = {
       #'C': [0.001, 0.01, 0.1, 1, 10],              # Regularization strength
        #'penalty': ['l2'],                           # Regularization type
        #'solver': ['liblinear', 'lbfgs'],           # Algorithm to use
        #'max_iter': [1000, 2000],                   # Maximum iterations
        #'tol': [1e-4, 1e-5]                         # Convergence tolerance
    #}

    clf = LogisticRegression(
        C=0.1,                # Optimal regularization strength
        penalty='l2',         # L2 regularization (Ridge)
        solver='lbfgs',       # Limited-memory BFGS algorithm
        max_iter=1000,        # Sufficient iterations for convergence
        tol=0.0001,           # Optimal tolerance for convergence
        random_state=42       # For reproducibility
    )

    # Train the model
    clf.fit(X_feat, y_train)

    # Return model weights and bias in required format
    return clf.coef_.reshape(-1), clf.intercept_[0]

In [ ]:
def my_map( X ):
################################
#  Non Editable Region Ending  #
################################

	# Use this method to create features.
	# It is likely that my_fit will internally call my_map to create features for train points
	# return feat
    X_pm = 2 * X - 1
    phi = np.cumprod(np.flip(X_pm, axis=1), axis=1)

    # Create indices for main diagonal and 2 off-diagonals using list comprehension
    diag_indices = [(i, j) for i in range(8) for j in range(i, min(i+3, 8))]

    # Convert list of tuples to two arrays for row and column indices
    row_idx, col_idx = zip(*diag_indices)

    # Use broadcasting to compute all features at once
    features = phi[:, row_idx] * phi[:, col_idx]

    # Apply scaling for faster convergence
    features_scaled = features / np.sqrt(np.mean(features**2, axis=1, keepdims=True) + 1e-10)

    return features_scaled

In [ ]:
def my_decode( w ):
################################
#  Non Editable Region Ending  #
################################

	# Use this method to invert a PUF linear model to get back delays
	# w is a single 65-dim vector (last dimension being the bias term)
	# The output should be four 64-dimensional vectors
	# -*- coding: utf-8 -*-
"""submit.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/18THH5wSVHLaF_oOO_HXdMy61NpoeiULr
"""
    """
    Takes a 65-dimensional linear model w as input and returns four
    64-dimensional non-negative delay vectors: p, q, r, s
    """

    # Ensure w is a 1D array
    w = np.asarray(w).flatten()

    # Separate out the bias term
    w_bias = w[-1]
    w_main = w[:-1]  # This is of shape (64,)

    # Initialize delay vectors
    alpha = np.copy(w_main)
    beta = np.copy(w_main)

    # Following the relation:
    # w = alpha + beta (excluding last element)
    # and w = alpha - beta (shifted version)

    # Solve for alpha and beta using equations:
    # alpha = (w[i] + w[i+1]) / 2
    # beta  = (w[i] - w[i+1]) / 2
    alpha = (w_main + np.roll(w_main, -1)) / 2
    beta = (w_main - np.roll(w_main, -1)) / 2

    # Remove the wrap-around artifact in last value
    alpha[-1] = w_main[-1] / 2
    beta[-1] = w_main[-1] / 2

    # Construct p, q, r, s from alpha and beta
    p = np.maximum(alpha, 0)
    q = np.maximum(-alpha, 0)
    r = np.maximum(beta, 0)
    s = np.maximum(-beta, 0)

    return p, q, r, s
	# return p, q, r, s
    return np.zeros(64), np.zeros(64), np.zeros(64), np.zeros(64)

In [ ]:
Z_trn = np.loadtxt( "/Users/ashutoshanand/Downloads/mp1/public_trn.txt" )
Z_tst = np.loadtxt( "/Users/ashutoshanand/Downloads/mp1/public_tst.txt" )

n_trials = 5

d_size = 0
t_train = 0
t_map = 0
acc = 0

In [ ]:
for t in range( n_trials ):
  tic = tm.perf_counter()
  w, b = my_fit( Z_trn[:, :-1], Z_trn[:,-1] )
  toc = tm.perf_counter()

  t_train += toc - tic
  w = w.reshape( -1 )

  d_size += w.shape[0]

  tic = tm.perf_counter()
  feat = my_map( Z_tst[:, :-1] )
  toc = tm.perf_counter()
  t_map += toc - tic

  scores = feat.dot( w ) + b

  pred = np.zeros_like( scores )
  pred[ scores > 0 ] = 1

  acc += np.average( Z_tst[ :, -1 ] == pred )

In [ ]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc /= n_trials

In [ ]:
def get_model( p, q, r, s ):
  p = np.maximum( p, 0 )
  q = np.maximum( q, 0 )
  r = np.maximum( r, 0 )
  s = np.maximum( s, 0 )
  d = p - q
  c = r - s
  alpha = ( d + c ) / 2
  beta = ( d - c ) / 2
  w = np.zeros( ( len( alpha ) + 1, )  )
  w[:-1] += alpha
  w[1:] += beta
  return w

In [ ]:
W = np.loadtxt( "/Users/ashutoshanand/Downloads/mp1/public_mod.txt" )
( n_models, dims ) = W.shape
t_decode = 0
m_dist = 0
for t in range( n_trials ):
  for itr in range( n_models ):
    w = W[ itr, : ]
    tic = tm.perf_counter()
    p_hat, q_hat, r_hat, s_hat = my_decode( w )
    toc = tm.perf_counter()
    t_decode += toc - tic
    w_hat = get_model( p_hat, q_hat, r_hat, s_hat )
    m_dist += np.linalg.norm( w - w_hat )

In [ ]:
t_decode /= ( n_trials * n_models )
m_dist /= ( n_trials * n_models )

In [ ]:
print( f"{d_size},{t_train},{t_map},{1 - acc},{t_decode},{m_dist}" )

21.0,0.023665116727352144,0.0024788502138108014,0.31375,5.726693198084831e-06,1.6199987057689555


In [ ]:
print( f"{d_size},{t_train},{t_map},{1 - acc},{t_decode},{m_dist}" )

21.0,0.013124933373183012,0.0006810663733631372,0.318125,2.7591409161686897e-06,1.6199987057689555
